In [16]:
from flask import Flask, request, jsonify
import mysql.connector
from mysql.connector import Error
from flask_cors import CORS


In [19]:

app = Flask(__name__)
CORS(app)

# Allowed tables and their corresponding columns
ALLOWED_TABLES = {
    "customerlocations": ["LocationID", "CustomerID", "City", "State", "Country", "PostalCode"],
    "customers": ["CustomerID", "CustomerName", "Segment"],
    "orderdetails": ["RowID", "OrderID", "ProductID", "Quantity", "ShippingCost", "OrderPriority"],
    "orders": ["OrderID", "OrderDate", "ShippingDate", "ShippingMode", "CustomerID"]
}

def get_database_connection():
    """Establish a connection to the MySQL database."""
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",      # MySQL Hostname
            user="root",           # MySQL Username
            password="q1w2e3r4",  # Replace with your root password
            database="globalsuperstore"  # Your database name
        )
        return conn
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        raise e

@app.route('/api/<table_name>/filter', methods=['GET'])
def filter_records(table_name):
    """Filter records based on field and value."""
    # Validate table name
    if table_name not in ALLOWED_TABLES:
        return jsonify({"error": f"Invalid table name '{table_name}'"}), 400

    # Get field and value from query parameters
    field = request.args.get('field')
    value = request.args.get('value')

    # Validate field
    if not field or field not in ALLOWED_TABLES[table_name]:
        return jsonify({"error": f"Invalid field '{field}' for table '{table_name}'"}), 400

    if not value:
        return jsonify({"error": "Missing 'value' parameter"}), 400

    try:
        conn = get_database_connection()
        cursor = conn.cursor(dictionary=True)  # Ensures results are returned as dictionaries
        query = f"SELECT * FROM {table_name} WHERE {field} = %s"
        cursor.execute(query, (value,))
        records = cursor.fetchall()
        conn.close()

        if not records:
            return jsonify({"message": "No records found"}), 404
        return jsonify(records)
    except Error as e:
        return jsonify({"status": "error", "message": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
DEBUG:mysql.connector:Building SSL context
DEBUG:mysql.connector:Switching to SSL
DEBUG:mysql.connector:SSL has been enabled
DEBUG:mysql.connector:Starting authorization phase
DEBUG:mysql.connector:The provided initial strategy is caching_sha2_password
DEBUG:mysql.connector:Exchanging further packets
DEBUG:mysql.connector:caching_sha2_password completed succesfully
127.0.0.1 - - [13/Jan/2025 15:09:58] "GET /api/customerlocations/filter?field=City&value=New%20York%20City HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/2025 15:09:58] "GET /api/customerlocations/filter?field=City&value=New%20York%20City HTTP/1.1" 200 -
DEBUG:mysql.connector:Building SSL context
DEBUG:mysql.connector:Switching to SSL
DEBUG:mysql.connector:SSL has